In [1]:
#export
import torch as _torch, torch.nn as _nn, numpy as _np
from functools import partial as _partial

In [2]:
#export
class Object:
    @staticmethod
    def listGenerator(): return []
    @staticmethod
    def zeroGenerator(): return 0
    def __init__(self):
        self._defaultValueGenerator = None
        self.repr = None
    @staticmethod
    def fromDict(dict):
        """Creates an object with attributes from a dictionary"""
        answer = Object(); answer.__dict__.update(dict); return answer
    def withAutoDeclare(self, defaultValueGenerator):
        """Sets this Object up so that if a field doesn't
        exist, it will automatically create it with a
        default value."""
        self._defaultValueGenerator = defaultValueGenerator
        return self
    def __getstate__(self): return self.__dict__
    def __setstate__(self, state): self.__dict__.update(state)
    def __getattr__(self, attr):
        if attr == "_defaultValueGenerator": raise AttributeError()
        if self._defaultValueGenerator != None:
            self.__dict__[attr] = self._defaultValueGenerator()
            return getattr(self, attr)
        raise AttributeError
    def withRepr(self, _repr):
        self.repr = _repr
        return self
    def __repr__(self):
        _dict = "".join([f"- {key}: {self.__dict__[key]}\n" for key in self.__dict__])
        return self.repr or f"<k1lib.basics.Object>, with __dict__:\n{_dict}"
class CancelRunException(Exception): pass
class CancelEpochException(Exception): pass
class CancelBatchException(Exception): pass
def test(value):
    """Tests if value is True"""
    assert value
def close(a, b):
    """Tests if 2 values are really close to each other"""
    test(_torch.allclose(_torch.tensor(a), _torch.tensor(b)))
def textToHtml(text:str) -> str:
    return text.replace("\n", "<br>").replace(" ", "&nbsp;")
def clearLine():
    print("\r" + " "*80 + "\r", end="")
def tab(text):
    """Adds a tab before each line"""
    return "\n".join(["    " + line for line in text.split("\n")])
def isNumeric(x):
    return type(x) == int or type(x) == float
def patch(_class, docs=None):
    def inner(_function):
        _docs = docs
        if _docs != None and type(_docs) is not str: _docs = _docs.__doc__
        _function.__doc__ = _docs or _function.__doc__ or _class.__doc__
        setattr(_class, _function.__qualname__, _function)
        return _function
    return inner
def _I(f): return f()
def I(f):
    """Returns `lambda: f()`"""
    return _partial(_I, f)

In [3]:
#export
class ListContainer:
    """Represents a list base class. Just for convenience's
    sake. Call _setField() to set the list field where the
    data will come from."""
    def __init__(self): self._setField("_items")
    def _setField(self, field):
        self.field = field
        setattr(self, self.field, [])
        self.ref = getattr(self, self.field)
        return self
    def __getitem__(self, idx): return self.ref[idx]
    def __setitem__(self, idx, value): self[idx] = value
    def __delitem__(self, idx): del(self[idx])
    def __len__(self): return len(getattr(self, self.field))
    def clear(self):
        self.ref.clear()
        return self
    def append(self, value):
        self.ref.append(value)
        return self

In [4]:
#export
class Range:
    def __init__(self, start=0, stop=None):
        if (isNumeric(start) and isNumeric(stop)):
            self.start = start
            self.stop = stop
        else:
            self.start = start[0]
            self.stop = start[1]
        self.delta = self.stop - self.start
    def __getitem__(self, index):
        if index == 0: return self.start
        if index == 1: return self.stop
        raise Exception(f"Can't get index {index} of range [{self.start}, {self.stop}]")
    def toUnit(self, x):
        return (x - self.start) / self.delta
    def fromUnit(self, x):
        return x * self.delta + self.start
    def toRange(self, _range, x):
        return Range(_range).fromUnit(self.toUnit(x))
    def __repr__(self):
        return f"""A range of numbers: [{self.start}, {self.stop}]. Can do:
- r.toUnit(x): will convert x from range [{self.start}, {self.stop}] to [0, 1]
- r.fromUnit(x): will convert x from range [0, 1] to range [{self.start}, {self.stop}]
- r.toRange([a, b], x): will convert x from range [{self.start}, {self.stop}] to range [a, b]
- r[0], r[1], r.start, r.stop: get start and stop values of range
"""

In [5]:
#export
def getFirst(x):
    """If `x` is a tuple or list, gets the first
    element. Else return the input."""
    return x[0] if type(x) in [list, tuple] else x

In [6]:
#export
def polyfit(x, y, deg):
    params = _np.polyfit(x, y, deg)
    def _inner(_x):
        answer = _np.zeros_like(_x, dtype=_np.float)
        for expo, param in enumerate(params):
            answer += param * _x**(len(params)-expo-1)
        return answer
    return _inner
def derivative(f, delta=1e-3):
    return lambda x: (f(x + delta) - f(x)) / delta
def optimize(f, v=1, threshold=1e-5):
    fD = derivative(f)
    while abs(f(v)) > threshold: v = v - f(v)/fD(v)
    return v

In [7]:
#export
def importParams(model:_nn.Module, params:list):
    for oldParam, newParam in zip(model.parameters(), params):
        oldParam.data = newParam.data.clone()
def exportParams(model:_nn.Module):
    return [param.data.clone() for param in model.parameters()]
def getParamsVector(model:_nn.Module):
    return [_torch.randn(param.shape).to(param.device) * (param.std() if len(param.squeeze().shape) > 0 else 1) for param in model.parameters()]

In [8]:
!../export.py _basics

Current dir: /home/kelvin/repos/labs/k1lib, ../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.1.0
Uninstalling k1lib-0.1.0:
  Successfully uninstalled k1lib-0.1.0
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/callbacks.py -> build/lib/k1lib
copying k1lib/data.py -> build/lib/k1lib
copying k1lib/_basics.py -> build/lib/k1lib
copying k1lib/nn.py -> build/lib/k1lib
copying k1lib/viz.py -> build/l

In [23]:
a = Object()

In [38]:
a = Object().withAutoDeclare(lambda: [])
setattr(a, "min", 3)
#a.min = 3

In [24]:
a.min = 3

In [25]:
a.min

3

In [34]:
a = Object(); a.min

AttributeError: 

In [32]:
a = Object.fromDict({"a": 3})

In [33]:
a.a

3